In [53]:
import os.path
dataset_path = r'/local/data1/elech646/Tumor_grade_classification/Dataset/HGG'

filepath = []
case_ids = []
patient_name = []
path_to_patient = []

for roots, dirs, files in os.walk("/local/data1/elech646/Tumor_grade_classification/Dataset"):
    for name in files:
        # Grab files larger than 30% in the name
        all_matches = re.findall('[0-9]*', name)
        perc = [int(i) for i in all_matches if i != ''][-1]
        if name.startswith('BraTS20') and perc >= 30:
            # '_'.join(name.split('_')[:3]) :17-20 number id
            filepath.append(roots + os.path.sep + name)
            case_ids.append(roots)
            path_to_patient.append(name)
            patient_name.append('_'.join(name.split('_')[:3]))
#         elif name.startswith('TCGA'):
#             filepath.append(roots + os.path.sep + name)
             #case_ids.append(roots)
#             patient_name.append('_'.join(name.split('_')[:1]))
                         
unique_patient_names = list(set(patient_name))

In [14]:
import os
import numpy as np
import re
import shutil

# Create folders for training/validation/test
model_data_path = '/local/data1/elech646/Tumor_grade_classification/dataset224'

if not os.path.exists(f'{model_data_path}/train'):
    os.mkdir(f'{model_data_path}/train')
    os.mkdir(f'{model_data_path}/train/G2')
    os.mkdir(f'{model_data_path}/train/G3')
    os.mkdir(f'{model_data_path}/train/G4')
    
if not os.path.exists(f'{model_data_path}/val'):
    os.mkdir(f'{model_data_path}/val')
    os.mkdir(f'{model_data_path}/val/G2')
    os.mkdir(f'{model_data_path}/val/G3')
    os.mkdir(f'{model_data_path}/val/G4')
    
if not os.path.exists(f'{model_data_path}/test'):
    os.mkdir(f'{model_data_path}/test')
    os.mkdir(f'{model_data_path}/test/G2')
    os.mkdir(f'{model_data_path}/test/G3')
    os.mkdir(f'{model_data_path}/test/G4')

In [15]:
#G4 = os.listdir("/local/data1/elech646/Tumor_grade_classification/Dataset (backup)/G4/frontal_grade_classification")
#G3 = os.listdir("/local/data1/elech646/Tumor_grade_classification/Dataset (backup)/LGG/frontal_grade_classification/G3")
#G2 = os.listdir("/local/data1/elech646/Tumor_grade_classification/Dataset (backup)/LGG/frontal_grade_classification/G2")

def train_val_test_split(grade):
    
    # Get always the same result
    np.random.seed(4)
    
    # Get 70 % as training (approx)
    train_idx = np.random.choice(grade, size = round(0.7*len(grade)), replace = False)
    
    # Find the non-training patients
    non_train = list(set(grade).difference(set(train_idx)))
    
    # Get 70 % of non-training as validation (approx)
    val_idx = np.random.choice(non_train, size = round(0.7*len(non_train)), replace = False)
    
    # Neither training nor validation => test
    test_idx = list(set(non_train).difference(set(val_idx)))
    
    return list(train_idx), list(val_idx), test_idx

# Uncomment for grade of choice

#G2_train, G2_val, G2_test = train_val_test_split(G2)
#G3_train, G3_val, G3_test = train_val_test_split(G3)
#G4_train, G4_val, G4_test = train_val_test_split(G4)

In [16]:
def get_images_40(grade_set, grade, new_path):

    if grade in ['G2', 'G3']:
        tumor_grade = 'LGG'
    else:
        tumor_grade = 'G4'
    
    for train_patient in grade_set:
        for m in ['frontal', 'sagittal', 'trans']:
            # for LGG cases
            if tumor_grade == 'LGG':
                path = f'/local/data1/elech646/Tumor_grade_classification/Dataset (backup)/{tumor_grade}/{m}_grade_classification/{grade}/{train_patient}'
            # for HGG cases
            else:
                path = f'/local/data1/elech646/Tumor_grade_classification/Dataset (backup)/{tumor_grade}/{m}_grade_classification/{train_patient}'
            # Get the names of all images
            all_images = os.listdir(path)
            for image in all_images:
                all_matches = re.findall('[0-9]*', image)
                perc = [int(i) for i in all_matches if i != ''][-1]
                if grade == 'G4':
                    front_cond = m == 'frontal'  and perc in range(12, 90+1)
                    sag_cond   = m == 'sagittal' and perc in range(13, 88+1)
                    trans_cond = m == 'trans'    and perc in range(17, 90+1)
                    if front_cond or sag_cond or trans_cond:
                        shutil.copyfile(f'{path}/{image}', f'{new_path}/{grade}/{image}')
                elif grade == 'G3':
                    front_cond = m == 'frontal'  and perc in range(6, 93+1)
                    sag_cond   = m == 'sagittal' and perc in range(16, 92+1)
                    trans_cond = m == 'trans'    and perc in range(13, 91+1)  
                    if front_cond or sag_cond or trans_cond:
                        shutil.copyfile(f'{path}/{image}', f'{new_path}/{grade}/{image}')
                else:
                    front_cond = m == 'frontal'  and perc in range(8, 93+1)
                    sag_cond   = m == 'sagittal' and perc in range(36, 96+1)
                    trans_cond = m == 'trans'    and perc in range(9, 92+1)  
                    if front_cond or sag_cond or trans_cond:
                        shutil.copyfile(f'{path}/{image}', f'{new_path}/{grade}/{image}')             

In [17]:
# Uncomment for the grade of choice

# G2
# get_images_40(G2_train, 'G2', '/local/data1/elech646/Tumor_grade_classification/dataset224/train')
# get_images_40(G2_val, 'G2', '/local/data1/elech646/Tumor_grade_classification/dataset224/val')
# get_images_40(G2_test, 'G2', '/local/data1/elech646/Tumor_grade_classification/dataset224/test')

# G3
# get_images_40(G3_train, 'G3', '/local/data1/elech646/Tumor_grade_classification/dataset224/train')
# get_images_40(G3_val, 'G3', '/local/data1/elech646/Tumor_grade_classification/dataset224/val')
# get_images_40(G3_test, 'G3', '/local/data1/elech646/Tumor_grade_classification/dataset224/test')

# G4
# get_images_40(G4_train, 'G4', '/local/data1/elech646/Tumor_grade_classification/dataset224/train')
# get_images_40(G4_val, 'G4', '/local/data1/elech646/Tumor_grade_classification/dataset224/val')
# get_images_40(G4_test, 'G4', '/local/data1/elech646/Tumor_grade_classification/dataset224/test')

In [ ]:
#for patient in set(case_ids):
for patient_path, image in zip(case_ids, path_to_patient):
    #re.findall("(?<=).*(?=\d*_\d*.png)", image)
    modality = re.sub(".+trans_|.+sag_|.+fro_|_\d+_\d+.png", "", image)
    #print(patient_path, modality)
    new_path = re.sub("(?<=/)(frontal.+/|sagittal.+/|trans.+/)(?=TCGA|BraTS)", "", patient_path)
    # Create folder for modality
    #if not os.path.exists(f"{new_path}/{modality}".replace("dataset", "dataset224")):
    #    os.mkdir(f"{new_path}".replace("dataset", "dataset224"))
    #    os.mkdir(f"{new_path}/{modality}".replace("dataset", "dataset224"))
    
    # Put image in modality folder
    #if not os.path.exists(f"{new_path}/{modality}/{image}".replace("dataset", "dataset224")):
        # FIX THIS
    #    shutil.copyfile(f"{patient_path}/{image}", 
    #                    f"{new_path}/{modality}/{image}".replace("dataset", "dataset224"))

In [63]:
patient_path = '/local/data1/elech646/Tumor_grade_classification/Dataset/HGG/trans_grade_classification/TCGA_Training_217/'
image = 'BraTS20_Training_217_fro_t1_46_52.png'
re.sub(".+trans_|.+sag_|.+fro_|_\d+_\d+.png", "", image)

't1'